In [ ]:
%pip install statsbombpy

In [ ]:
# 01_data_exploration.ipynb

# Installazione (se non già fatto)
# %pip install statsbombpy

from statsbombpy import sb
import pandas as pd

# 1. Lista delle competizioni disponibili
competitions = sb.competitions()
print("Competizioni disponibili:")
display(competitions)

# 2. Filtraggio competizioni con freeze frame disponibili
# (StatsBomb fornisce freeze frame solo per alcune partite, es. WSL, FAWSL, ecc.)
# Noi qui per Task1 cerchiamo tutte quelle che hanno "Women's Super League" o "FAWSL"
# oppure includiamo quelle che vogliamo analizzare
competitions_with_ff = competitions[
    competitions['competition_name'].str.contains("Women's|FA Women's|FAWSL|World Cup", case=False, na=False)
]
print("Competizioni candidate con freeze frame:")
display(competitions_with_ff)

# 3. Scegli una competizione e stagione
comp_id = int(competitions_with_ff.iloc[0]['competition_id'])
season_id = int(competitions_with_ff.iloc[0]['season_id'])

print(f"\nScarico matches per {competitions_with_ff.iloc[0]['competition_name']} - {competitions_with_ff.iloc[0]['season_name']}")
matches = sb.matches(competition_id=comp_id, season_id=season_id)
display(matches.head())

# 4. Prendo un match e carico gli eventi
match_id = matches.iloc[0]['match_id']
events = sb.events(match_id=match_id, fmt='dataframe')
display(events.head())

# 5. Filtro solo i tiri
shots = events[events['type'] == 'Shot']
print(f"Numero di tiri nel match {match_id}: {len(shots)}")
display(shots.head())

# 6. Controllo presenza del freeze frame
# La colonna 'shot_freeze_frame' contiene i dati di tracking dei giocatori vicini alla palla
has_ff = shots['shot_freeze_frame'].notna().sum()
print(f"Tiri con freeze frame disponibile: {has_ff} su {len(shots)}")

# Visualizzo un esempio di freeze frame (lista di dict)
example_ff = shots.iloc[0]['shot_freeze_frame']
print("Esempio freeze frame:")
example_ff[:3]  # primi 3 giocatori nel freeze frame
